In [ ]:
from nbdev import *
%nbdev_default_export hierarchy

Cells will be exported to pct.hierarchy,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

In [ ]:
# hide
import sys
sys.path.append("..")

# Hierarchy

Creation of Perceptual Control hierarchies.

In [ ]:
%nbdev_export
import numpy as np
from pct.nodes import PCTNode
from pct.functions import *

In [ ]:
%nbdev_export
class PCTHierarchy():
    "A hierarchical perceptual control system, of PCTNodes."
    def __init__(self, rows=1, cols=1, name="pcthierarchy", links="single", history=False, **pargs):
        self.links_built = False
        UniqueNamer.getInstance().clear()
        self.name=UniqueNamer.getInstance().get_name(name)
        
        self.hierarchy = []
        print(self.hierarchy)
        for r in range(rows):
            col_list=[]
            for c in range(cols):
                if links == "dense":
                    perc = WeightedSum(weights=np.ones(cols))
                    node = PCTNode(perception=perc, name=f'row{r}col{c}')
                else:
                    node = PCTNode(name=f'row{r}col{c}')
                self.handle_links(node, r, c, links)
                col_list.append(node)
                
            self.hierarchy.append(col_list)
                

    def handle_links(self, node, row, col, links_type):
        if row == 0 or links_type == None:
            return
        
        if links_type == "single":
            node.add_link("perception", self.hierarchy[row-1][col].get_function("perception"))
        
        if links_type == "dense":
            for column in range(len(self.hierarchy[row-1])):
                node.add_link("perception", self.hierarchy[row-1][column].get_function("perception"))
            
    def get_config(self):
        config = {"type": type(self).__name__,
                    "name": self.name}        
        
        levels = {}
        for row in range(len(self.hierarchy)):
            level ={'level':row}
            columns={}
            for col in range(len(self.hierarchy[row])):
                column={'col':col}
                nodeconfig = self.hierarchy[row][col].get_config()
                #print(nodeconfig)
                column['node']=nodeconfig
                #print(column)
                columns[f'col{col}']=column
            level['nodes']=columns
            levels[f'level{row}']=level
        config['levels']=levels
        return config       


In [ ]:
hpct = PCTHierarchy(3,3, links="dense")
hpct.hierarchy


[]


[[<pct.nodes.PCTNode at 0x215f106aa08>,

In [ ]:
hpct.get_config()

{'type': 'PCTHierarchy',
 'name': 'pcthierarchy',
 'levels': {'level0': {'level': 0,
   'nodes': {'col0': {'col': 0,
     'node': {'type': 'PCTNode',
      'name': 'row0col0',
      'refcoll': {'0': {'type': 'Constant',
        'name': 'constant',
        'value': 1,
        'links': {}}},
      'percoll': {'0': {'type': 'WeightedSum',
        'name': 'weighted_sum',
        'value': 0,
        'links': {}}},
      'comcoll': {'0': {'type': 'Subtract',
        'name': 'subtract',
        'value': 0,
        'links': {}}},
      'outcoll': {'0': {'type': 'Proportional',
        'name': 'proportional',
        'value': 0,
        'links': {},
        'gain': 10}}}},
    'col1': {'col': 1,
     'node': {'type': 'PCTNode',
      'name': 'row0col1',
      'refcoll': {'0': {'type': 'Constant',
        'name': 'constant1',
        'value': 1,
        'links': {}}},
      'percoll': {'0': {'type': 'WeightedSum',
        'name': 'weighted_sum1',
        'value': 0,
        'links': {}}},
      

In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
